In [2]:
import boto3
import sagemaker

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
from sagemaker.session import TrainingInput
from sagemaker import image_uris
from sagemaker import hyperparameters

In [4]:
import logging

In [5]:
boto3.set_stream_logger(name='botocore.credentials', level=logging.WARNING)

In [6]:
region = sagemaker.Session().boto_region_name
print(region)

us-east-1


In [7]:
import os

In [8]:
# role_arn = os.getenv('SGMKR_ROLE_ARN')
role_arn = sagemaker.get_execution_role()

In [9]:
print(role_arn)

arn:aws:iam::216393269487:role/service-role/AmazonSageMaker-ExecutionRole-20240925T153897


In [10]:
bucket = 'sagemakerlearningwithiris'
prefix = 'iris'

In [11]:
#Get the ls of objects in the Bucket with name bucket and prefix
!aws s3 ls {bucket}/{prefix}/

                           PRE batch_transform/
                           PRE data/
                           PRE model/


In [12]:
!aws s3 ls {bucket}/{prefix}/data/ --recursive

2024-09-25 19:10:22        540 iris/data/iris_test.csv
2024-09-25 19:10:21       2160 iris/data/iris_train.csv


In [13]:
train_file = 'data/iris_train.csv'
validation_file = 'data/iris_test.csv'

train_file_uri = 's3://{}/{}/{}'.format(bucket, prefix, train_file)
test_file_uri = 's3://{}/{}/{}'.format(bucket, prefix, validation_file)
print(train_file_uri)
print(test_file_uri)

s3://sagemakerlearningwithiris/iris/data/iris_train.csv
s3://sagemakerlearningwithiris/iris/data/iris_test.csv


In [14]:
train_ip = TrainingInput(train_file_uri, content_type='csv')
test_ip = TrainingInput(test_file_uri, content_type='csv')
print(train_ip)
print(test_ip)

In [15]:
model_op = 's3://{}/{}/{}'.format(bucket, prefix, 'model')
print(model_op)

s3://sagemakerlearningwithiris/iris/model


In [16]:
model_img = sagemaker.image_uris.retrieve('xgboost', region, 'latest')
print(model_img)

811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest


In [17]:
job_name = 'iris-xgboost'
xgb_model = sagemaker.estimator.Estimator(
    image_uri=model_img,
    role = role_arn,
    base_job_name=job_name,
    instance_count=1,
    instance_type='ml.m5.4xlarge',
    output_path=model_op,
    sagemaker_session=sagemaker.Session(),
    volume_size=5
)

In [18]:
# xgb_model.set_hyperparameters(num_class=3, max_depth=5, num_round=10, objective='multi:softmax')
# xgb_model.set_hyperparameters(num_class=3, max_depth=5, num_round=10, objective='multi:softmax')
xgb_model.set_hyperparameters(num_class=3, max_depth=5, num_round=10, objective='multi:softmax')


In [19]:
from datetime import datetime
name_job = job_name + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(name_job)

iris-xgboost2024-09-26-04-41-31


In [20]:
xgb_model.fit({"train":train_ip, "validation":test_ip}, wait=True, job_name=name_job)

INFO:sagemaker:Creating training-job with name: iris-xgboost2024-09-26-04-41-31


2024-09-26 04:41:31 Starting - Starting the training job...
2024-09-26 04:41:52 Starting - Preparing the instances for training...
2024-09-26 04:42:31 Downloading - Downloading input data...
2024-09-26 04:42:51 Downloading - Downloading the training image...
2024-09-26 04:43:22 Training - Training image download completed. Training in progress..Arguments: train
[2024-09-26:04:43:40:INFO] Running standalone xgboost training.
[2024-09-26:04:43:40:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 55446.71mb
[2024-09-26:04:43:40:INFO] Determined delimiter of CSV input is ','
[04:43:40] S3DistributionType set as FullyReplicated
[04:43:40] 120x4 matrix with 480 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-09-26:04:43:40:INFO] Determined delimiter of CSV input is ','
[04:43:40] S3DistributionType set as FullyReplicated
[04:43:40] 30x4 matrix with 120 entries loaded from /opt/ml/input/data/validation?format=c

In [21]:
!aws s3 ls s3://{bucket}/{prefix}/model/

                           PRE iris-xgboost2024-09-25-19-02-02/
                           PRE iris-xgboost2024-09-25-19-15-18/
                           PRE iris-xgboost2024-09-26-04-41-31/


In [22]:
!aws s3 ls s3://{bucket}/{prefix}/model/{name_job}/

                           PRE debug-output/
                           PRE output/
                           PRE profiler-output/


## Model Deployment and Making Inference

#### -  CSVserializer from sagemaker is used to serialize the inference data into a formart(CSV) in which the model training data is 

In [23]:
from sagemaker.serializers import CSVSerializer
serializer = CSVSerializer()

In [24]:
type(xgb_model)

sagemaker.estimator.Estimator

#### The Below function is used ti deploy the XGBoost model to an endpoint
- Initial_instance_count is the number of instance (Virtual Machines) to host the endpoint and serve the request
- CSVserializer = is used to convert the data that is needed to make inference in formart that model expects
- instance_type = Type of instance that will be used to host the model

In [26]:
xgb_predictor = xgb_model.deploy(
    initial_instance_count=1, instance_type='ml.t2.medium', serializer=CSVSerializer()
)

INFO:sagemaker:Creating model with name: iris-xgboost-2024-09-26-04-46-15-694
INFO:sagemaker:Creating endpoint-config with name iris-xgboost-2024-09-26-04-46-15-694
INFO:sagemaker:Creating endpoint with name iris-xgboost-2024-09-26-04-46-15-694


------!

In [27]:
xgb_predictor.predict('7.7, 3.0, 6.1, 2.3')

b'2.0'

In [29]:
endpoint_name = xgb_predictor.endpoint_name
print(endpoint_name)

iris-xgboost-2024-09-26-04-46-15-694


In [32]:
payload_csv_text =" 7.7, 3.0, 6.1, 2.3"
sgmaker_runtime = boto3.client("runtime.sagemaker") #Creates sagemaker runtime client that is used to invoke deployed sagemaker endpoints to get inference
response = sgmaker_runtime.invoke_endpoint(
    EndpointName = endpoint_name, ContentType='text/csv', Body=payload_csv_text
)

In [33]:
print(response)

{'ResponseMetadata': {'RequestId': '787755d5-ca45-4a04-86aa-f6a0bbc08e08', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '787755d5-ca45-4a04-86aa-f6a0bbc08e08', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Thu, 26 Sep 2024 05:09:14 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '3', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f6dd0bc4d00>}


In [34]:
print(response["Body"].read().decode())

2.0


## Endpoint- Multiple Inference

In [37]:
csv_buffer = open("Data/iris_infer.csv")
payload_csv_text = csv_buffer.read()

In [38]:
response = sgmaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType='text/csv', Body=payload_csv_text
)
print(response["Body"].read().decode())

1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0


In [39]:
payload_csv_text

'5.4,3.0,4.5,1.5\n5.8,2.7,5.1,1.9\n6.5,3.0,5.5,1.8\n6.5,3.0,5.8,2.2\n6.0,2.2,5.0,1.5\n6.6,3.0,4.4,1.4\n6.7,3.3,5.7,2.1\n6.3,3.3,4.7,1.6\n5.2,3.5,1.5,0.2\n6.4,3.1,5.5,1.8\n6.4,2.9,4.3,1.3\n5.6,2.5,3.9,1.1\n4.9,3.6,1.4,0.1\n5.4,3.4,1.5,0.4\n7.2,3.2,6.0,1.8\n5.5,2.4,3.7,1.0\n5.4,3.7,1.5,0.2\n4.9,3.0,1.4,0.2\n5.0,2.3,3.3,1.0\n4.8,3.0,1.4,0.1\n6.5,3.2,5.1,2.0\n6.0,3.4,4.5,1.6\n6.2,2.9,4.3,1.3\n6.1,2.6,5.6,1.4\n5.1,3.8,1.5,0.3\n4.6,3.4,1.4,0.3\n7.3,2.9,6.3,1.8\n4.6,3.2,1.4,0.2\n5.0,3.4,1.5,0.2\n5.7,2.6,3.5,1.0\n'

## Endpoint - Multiple Endpoint from S3 File

In [40]:
print(bucket)

sagemakerlearningwithiris


In [41]:
infer_ip_uri = "s3://{}/{}/{}".format(bucket, prefix, "batch_transform/iris_infer.csv")
print(infer_ip_uri)

s3://sagemakerlearningwithiris/iris/batch_transform/iris_infer.csv


In [47]:
import pandas as pd

In [50]:
s3_clnt = boto3.client("s3")
obj = s3_clnt.get_object(Bucket=bucket, Key="iris/batch_transform/iris_infer.csv")
payload_df = pd.read_csv(obj["Body"])
# print(obj)

In [51]:
payload_df.head()

,5.4,3.0,4.5,1.5
0,5.8,2.7,5.1,1.9
1,6.5,3.0,5.5,1.8
2,6.5,3.0,5.8,2.2
3,6.0,2.2,5.0,1.5
4,6.6,3.0,4.4,1.4


In [56]:
import io
csv_buffer = io.StringIO() # Creates an in memory string buffer that acts like a file object
payload_df.to_csv(csv_buffer, header=None, index=None)  # Convert a payload df and writes it into a csv_buffer
payload_csv_text =  csv_buffer.getvalue() # Retrieve the content from buffer
response = sgmaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
print(response["Body"].read().decode())


2.0,2.0,2.0,1.0,1.0,2.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0


## Delete the endpoint

In [58]:
sgm_clnt = boto3.client("sagemaker")
sgm_clnt.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '1dd1e9c6-43b0-492f-a234-eb96a6b83711',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1dd1e9c6-43b0-492f-a234-eb96a6b83711',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 26 Sep 2024 05:58:36 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

## Batch Transform

In [59]:
batch_ip = "s3://{}/{}/{}".format(bucket, prefix, "batch_transform")
batch_op = "s3://{}/{}/{}".format(bucket, prefix, "batch_transform")

In [60]:
print(batch_ip)

s3://sagemakerlearningwithiris/iris/batch_transform


In [62]:
!aws s3 ls {batch_ip} --recursive

2024-09-25 19:10:23        480 iris/batch_transform/iris_infer.csv


In [63]:
transformer = xgb_model.transformer(
    instance_count=1, instance_type="ml.m4.xlarge", output_path=batch_op
)

INFO:sagemaker:Creating model with name: iris-xgboost-2024-09-26-06-03-32-447


In [64]:
print(transformer)

In [65]:
transformer.transform(
    data=batch_ip, data_type="S3Prefix", content_type="text/csv", split_type="Line"
)
transformer.wait()

INFO:sagemaker:Creating transform job with name: iris-xgboost-2024-09-26-06-07-32-720


.......................................
Arguments: serve
[2024-09-26 06:14:07 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2024-09-26 06:14:07 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2024-09-26 06:14:07 +0000] [1] [INFO] Using worker: gevent
[2024-09-26 06:14:07 +0000] [21] [INFO] Booting worker with pid: 21
[2024-09-26 06:14:07 +0000] [22] [INFO] Booting worker with pid: 22
/opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util.ssl_ (/opt/amazon/lib/python3.7/site-packages/urllib3/util/ssl_.py)', 'urllib3.util (/opt/amazon/lib/python3.7/site-packages/urllib3/util/__init__.py)']. 
  monkey.patch_all(subprocess=True)
[2

In [66]:
!aws s3 ls s3://{bucket}/{prefix}/batch_transform/ --recursive

2024-09-25 19:10:23        480 iris/batch_transform/iris_infer.csv
2024-09-26 06:14:11        120 iris/batch_transform/iris_infer.csv.out


In [72]:
!aws s3 cp s3://{bucket}/{prefix}/batch_transform/iris_infer.csv.out . 

download: s3://sagemakerlearningwithiris/iris/batch_transform/iris_infer.csv.out to ./iris_infer.csv.out


In [74]:
!head  -n 5iris_infer.csv.out

head: 5iris_infer.csv.out: invalid number of lines
